In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
from itertools import combinations
import keras

In [ ]:
from IPython.display import display
#import kaleido

In [ ]:
import pandas as pd
import numpy as np
Vehicle = pd.read_csv(#NHTS Vehicle File)
Household=pd.read_csv(#NHTS Household File)
Pop=pd.read_csv('#US Census household by state')

In [ ]:
#https://www.epa.gov/greenvehicles/greenhouse-gas-rating#:~:text=This%20rating%20reflects%20vehicle%20tailpipe,tailpipe%20emissions%20of%20greenhouse%20gases.

def calculate_new_column_value(row):
    veh_year = row['VEHYEAR']
    hfuel = row['HFUEL']
    fegempg = row['FEGEMPG']
    fegempga = row['FEGEMPGA']

    if veh_year <= 2008:
        if hfuel == 2:
            return 10779 * fegempga ** -0.994
        else:
            return 10779 * fegempg ** -0.994
    elif veh_year in [2009, 2010]:
        if hfuel == 2:
            return 8858.7 * fegempga ** -0.996
        else:
            return 8858.7 * fegempg ** -0.996
    elif veh_year in [2011, 2012]:
        if hfuel == 2:
            return 8770.9 * fegempga ** -0.993
        else:
            return 8770.9 * fegempg ** -0.993
    elif veh_year == 2013:
        if hfuel == 2:
            return 7954.4 * fegempga ** -0.972
        else:
            return 7954.4 * fegempg ** -0.972
    elif veh_year == 2014:
        if hfuel == 2:
            return 8710.2 * fegempga ** -0.993
        else:
            return 8710.2 * fegempg ** -0.993
    elif veh_year == 2015:
        if hfuel == 2:
            return 8748.6 * fegempga ** -0.995
        else:
            return 8748.6 * fegempg ** -0.995
    elif veh_year == 2016:
        if hfuel == 2:
            return 8760.5 * fegempga ** -0.995
        else:
            return 8760.5* fegempg ** -0.995
    elif veh_year == 2017:
        if hfuel == 2:
            return 8762.4 * fegempga ** -0.995
        else:
            return 8762.4 * fegempg ** -0.995
    # Add more conditions for other years as needed
    else:
        return 0  # Default value if year is not specified or condition not met

In [ ]:
Vehicle['Gas_Emission'] = Vehicle.apply(lambda row: "{:.2f}".format(calculate_new_column_value(row)), axis=1)

In [ ]:
# Remove rows where VEHYEAR is -8 or -7
Vehicle = Vehicle.loc[~Vehicle['VEHYEAR'].isin([-8, -7])]

# Reset the index if desired
Vehicle.reset_index(drop=True, inplace=True)

In [ ]:
Vehicle.loc[Vehicle['HFUEL'].isin([2,3]), 'Gas_Emission'] = 0

In [ ]:
# Remove rows where BESTMILE is -9 or 0
Vehicle = Vehicle[(Vehicle['BESTMILE'] != -9) & (Vehicle['BESTMILE'] != 0)]

# Reset the index of the modified dataframe
Vehicle.reset_index(drop=True, inplace=True)

In [ ]:
# Convert columns to numeric data types
Vehicle['BESTMILE'] = pd.to_numeric(Vehicle['BESTMILE'], errors='coerce')
Vehicle['Gas_Emission'] = pd.to_numeric(Vehicle['Gas_Emission'], errors='coerce')

Non-numeric values in BESTMILE column:
Series([], Name: BESTMILE, dtype: float64)

Non-numeric values in Gas_Emission column:
Series([], Name: Gas_Emission, dtype: float64)


<ipython-input-14-d612e11cf9b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vehicle['BESTMILE'] = pd.to_numeric(Vehicle['BESTMILE'], errors='coerce')
<ipython-input-14-d612e11cf9b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vehicle['Gas_Emission'] = pd.to_numeric(Vehicle['Gas_Emission'], errors='coerce')


In [ ]:
# https://www.fueleconomy.gov/feg/label/calculations-information.shtml
#https://www.fueleconomy.gov/feg/Find.do?action=bt1

# Create the Annual_emission column
Vehicle['Gasoline_emission'] = Vehicle['BESTMILE'] * Vehicle['Gas_Emission']/(1000*1000)
Vehicle['Annual_emission_upstream'] = 0.25*Vehicle['Gasoline_emission']
Vehicle['Annual_emission'] = Vehicle['Gasoline_emission']+Vehicle['Annual_emission_upstream']
#1gallon=33.7KWH
#119MPGe=33.7KWH
#100mile=xKWH?
#x=33.7*100/119=28KWH/100 Mile = 0.2831KWH/mile = 0.2831*0.001 MWH/mile
# Create a new column 'EV_upstream_emission' with the calculated values
Vehicle['Max_EV_upstream_emission'] = 33.7 * Vehicle['BESTMILE'] * 100 * 0.001 * 0.01/80
Vehicle['Min_EV_upstream_emission'] = 33.7 * Vehicle['BESTMILE'] * 100 * 0.001 * 0.01/100
# # Apply the if-else condition
# Vehicle['EV_upstream_emission'] = Vehicle.apply(lambda row: row['EV_upstream_emission'] / row['FEGEMPGA']
#                                                if row['HFUEL'] != 3
#                                                else row['EV_upstream_emission'] / row['FEGEMPG'],
#                                                axis=1)
# Display the updated dataset
# print(Vehicle)

<ipython-input-15-546ef669b293>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vehicle['Gasoline_emission'] = Vehicle['BESTMILE'] * Vehicle['Gas_Emission']/(1000*1000)
<ipython-input-15-546ef669b293>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vehicle['Annual_emission_upstream'] = 0.25*Vehicle['Gasoline_emission']
<ipython-input-15-546ef669b293>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [ ]:
# Load eGRID 2021 summary data
elec = pd.read_csv()  # eGRID2021 summary tables

# Load projection data for 2035
elec2 = pd.read_excel()  # Midcase 2035 projection

# Load projection data for 2050
elec3 = pd.read_excel()  # Midcase 2050 projection

# Load projection data for 2035 with M95 scenario
elec4 = pd.read_excel()  # 2035 M95 scenario

# Load projection data for 2050 with M95 scenario
elec5 = pd.read_excel()  # 2050 M95 scenario

# Load projection data for 2035 with LREC scenario
elec6 = pd.read_excel()  # 2035 LREC scenario

# Load projection data for 2050 with LREC scenario
elec7 = pd.read_excel()  # 2050 LREC scenario


In [ ]:
import pandas as pd
# Rename state columns for merging consistency
elec.rename(columns={'HHSTATE': 'State'}, inplace=True)
for df, name in zip([elec2, elec3, elec4, elec5, elec6, elec7],
                    ['2035', '2050', '2035_M95', '2050_M95', '2035_LREC', '2050_LREC']):
    df.rename(columns={'state': 'State', 'aer_gen_co2e': name}, inplace=True)

# Select relevant columns
elec = elec[['State', 'CO2e']]
elec2 = elec2[['State', '2035']]
elec3 = elec3[['State', '2050']]
elec4 = elec4[['State', '2035_M95']]
elec5 = elec5[['State', '2050_M95']]
elec6 = elec6[['State', '2035_LREC']]
elec7 = elec7[['State', '2050_LREC']]

# Merge all files on 'State'
merged_df1 = elec
for df in [elec2, elec3, elec4, elec5, elec6, elec7]:
    merged_df1 = merged_df1.merge(df, on='State', how='outer')

# Rename 'State' back to 'HHSTATE'
merged_df1.rename(columns={'State': 'HHSTATE'}, inplace=True)


In [ ]:
merged_df1['CO2e'] = merged_df1['CO2e'].str.replace(',', '').astype(float)

In [ ]:
merged_df = pd.merge(Vehicle, merged_df1, on='HHSTATE')

In [ ]:
#0.0005 = lb to ton 0.001 is converting KG/MWH to KG/KWH
merged_df['Min_EV_upstream_emission_total'] = merged_df['Min_EV_upstream_emission']*merged_df['CO2e']*0.000453592
merged_df['Max_EV_upstream_emission_total'] = merged_df['Max_EV_upstream_emission']*merged_df['CO2e']*0.000453592
merged_df['MC_2035'] = merged_df['Min_EV_upstream_emission']*merged_df['2035']*0.001
merged_df['MC_2050'] = merged_df['Min_EV_upstream_emission']*merged_df['2050']*0.001
merged_df['MC_2035_M95'] = merged_df['Min_EV_upstream_emission']*merged_df['2035_M95']*0.001
merged_df['MC_2050_M95'] = merged_df['Min_EV_upstream_emission']*merged_df['2050_M95']*0.001
merged_df['LREC_2035_LREC'] = merged_df['Min_EV_upstream_emission']*merged_df['2035_LREC']*0.001
merged_df['LREC_2050_LREC'] = merged_df['Min_EV_upstream_emission']*merged_df['2050_LREC']*0.001

In [ ]:
# Assuming your filtered data is in a DataFrame called 'filtered_df'

# List of columns to keep in the new DataFrame
keep_columns = ['HFUEL', 'FUELTYPE', 'FEGEMPGA', 'HHFAMINC','HHSTATE','WTHHFIN','HBHUR']

# Create a new DataFrame by grouping and summarizing the data
summary_df = merged_df.groupby('HOUSEID').agg({
    'Annual_emission': 'sum',
    'Min_EV_upstream_emission_total': 'sum',
    'Max_EV_upstream_emission_total': 'sum',
    'MC_2035': 'sum',
    'MC_2050': 'sum',
    'MC_2035_M95': 'sum',
    'MC_2050_M95': 'sum',
    'LREC_2035_LREC': 'sum',
    'LREC_2050_LREC': 'sum',
    'FEGEMPG':'sum',
    'BESTMILE': 'sum',
    **{column: 'first' for column in keep_columns}
}).reset_index()


In [ ]:
summary_df.loc[summary_df['HFUEL'].isin([2,3]), 'Annual_emission'] = 0
summary_df.loc[summary_df['HFUEL'].isin([-1,4]), 'EV_upstream_emission_total'] = 0

In [ ]:
summary_df['Household_emission'] = summary_df['Annual_emission']+summary_df['EV_upstream_emission_total']

In [ ]:
merged_df2 = pd.merge(Household,summary_df, on='HOUSEID', how='left').fillna(0)

In [ ]:
#Include weights

merged_df2['Weighted_EVemission'] =merged_df2['Max_EV_upstream_emission_total']*merged_df2['WTHHFIN_x']
merged_df2['Weighted_MC_2035'] = merged_df2['MC_2035'] * merged_df2['WTHHFIN_x']
merged_df2['Weighted_MC_2050'] = merged_df2['MC_2050'] * merged_df2['WTHHFIN_x']
merged_df2['Weighted_MC_2035_M95'] = merged_df2['MC_2035_M95'] * merged_df2['WTHHFIN_x']
merged_df2['Weighted_MC_2050_M95'] = merged_df2['MC_2050_M95'] * merged_df2['WTHHFIN_x']
merged_df2['Weighted_LREC_2035'] = merged_df2['LREC_2035_LREC'] * merged_df2['WTHHFIN_x']
merged_df2['Weighted_LREC_2050'] = merged_df2['LREC_2050_LREC'] * merged_df2['WTHHFIN_x']
merged_df2 ['Weighted_emission'] =merged_df2 ['Household_emission']*merged_df2 ['WTHHFIN_x']
merged_df2 ['Weighted_Miles'] =merged_df2 ['BESTMILE']*merged_df2['WTHHFIN_x']
merged_df2 ['Weighted_efficiency'] =merged_df2 ['FEGEMPG']*merged_df2['WTHHFIN_x']



In [ ]:
summary_df=merged_df2[['HHFAMINC_x','HHSTATE_x','WTHHFIN_x','HBHUR_x','HOUSEID','Household_emission','BESTMILE','FEGEMPG','Weighted_emission','Weighted_Miles','Weighted_efficiency','Weighted_EVemission','Weighted_MC_2035','Weighted_MC_2050','Weighted_MC_2035_M95','Weighted_MC_2050_M95','Weighted_LREC_2035','Weighted_LREC_2050']]
#summary_df=merged_df2[['HHFAMINC_x','HHSTATE_x','WTHHFIN_x','HBHUR_x','HOUSEID','BESTMILE','FEGEMPG','Weighted_Miles','Weighted_efficiency','Weighted_EVemission','Weighted_MC_2035','Weighted_MC_2050','Weighted_MC_2035_M95','Weighted_MC_2050_M95','Weighted_LREC_2035','Weighted_LREC_2050']]

In [ ]:
grouped_data = summary_df.groupby('HHSTATE_x').agg({
    'Weighted_Miles': 'sum',
    'WTHHFIN_x': 'sum',
    'Weighted_EVemission': 'sum',
    'Weighted_emission': 'sum',
    'Weighted_MC_2035': 'sum',
    'Weighted_MC_2050': 'sum',
    'Weighted_MC_2035_M95': 'sum',
    'Weighted_MC_2050_M95': 'sum',
    'Weighted_LREC_2035': 'sum',
    'Weighted_LREC_2050': 'sum',
})

In [ ]:
# Grouping the data by HHSTATE and calculating the sum of relevant columns
#grouped_data = summary_df.groupby('HHSTATE_x').agg({'Weighted_Miles': 'sum','WTHHFIN_x': 'sum', 'Weighted_EVemission': 'sum', 'Weighted_emission': 'sum'})

# Calculating weighted miles per household
grouped_data['Weighted_Miles_Per_Household'] = grouped_data['Weighted_Miles'] / grouped_data['WTHHFIN_x']

# Calculating weighted miles per household
grouped_data['Weighted_EVemission2'] = grouped_data['Weighted_EVemission'] / grouped_data['WTHHFIN_x']

# # Calculating average efficiency
# grouped_data['Average_Efficiency'] = grouped_data['Weighted_efficiency'] / grouped_data['WTHHFIN']

# Calculating average emissions
grouped_data['Average_Emissions'] = grouped_data['Weighted_emission'] / grouped_data['WTHHFIN_x']

#grouped_data['Max_change']=abs((grouped_data['Weighted_EVemission2']-grouped_data['Average_Emissions'])/grouped_data['Average_Emissions'])*100

grouped_data['Weighted_MC_2035_2'] = grouped_data['Weighted_MC_2035'] / grouped_data['WTHHFIN_x']
grouped_data['Weighted_MC_2050_2'] = grouped_data['Weighted_MC_2050'] / grouped_data['WTHHFIN_x']
grouped_data['Weighted_MC_2035_M95_2'] = grouped_data['Weighted_MC_2035_M95'] / grouped_data['WTHHFIN_x']
grouped_data['Weighted_MC_2050_M95_2'] = grouped_data['Weighted_MC_2050_M95'] / grouped_data['WTHHFIN_x']
grouped_data['Weighted_LREC_2035_2'] = grouped_data['Weighted_LREC_2035'] / grouped_data['WTHHFIN_x']
grouped_data['Weighted_LREC_2050_2'] = grouped_data['Weighted_LREC_2050'] / grouped_data['WTHHFIN_x']

In [ ]:
merged_df3 = pd.merge(grouped_data,Pop, on='HHSTATE_x')

In [ ]:
merged_df3['Total_emission']=merged_df3['Average_Emissions']*merged_df3['Household_No.']
merged_df3['Total_EVemission']=merged_df3['Weighted_EVemission2']*merged_df3['Household_No.']
merged_df3['Max_change_percentage']=abs((merged_df3['Total_EVemission']-merged_df3['Total_emission'])/merged_df3['Total_emission'])*100
merged_df3['Max_change_percentage_EV']=abs((merged_df3['Weighted_EVemission2']-merged_df3['Average_Emissions'])/merged_df3['Average_Emissions'])*100
merged_df3['Max_change']=abs((merged_df3['Total_emission']-merged_df3['Total_EVemission']))
merged_df3['Total_MC_2035'] = merged_df3['Weighted_MC_2035_2'] * merged_df3['Household_No.']
merged_df3['Total_MC_2050'] = merged_df3['Weighted_MC_2050_2'] * merged_df3['Household_No.']
merged_df3['Total_MC_2035_M95'] = merged_df3['Weighted_MC_2035_M95_2'] * merged_df3['Household_No.']
merged_df3['Total_MC_2050_M95'] = merged_df3['Weighted_MC_2050_M95_2'] * merged_df3['Household_No.']
merged_df3['Total_LREC_2035'] = merged_df3['Weighted_LREC_2035_2'] * merged_df3['Household_No.']
merged_df3['Total_LREC_2050'] = merged_df3['Weighted_LREC_2050_2'] * merged_df3['Household_No.']



,HHSTATE_x,Weighted_Miles,WTHHFIN_x,Weighted_EVemission,Weighted_emission,Weighted_MC_2035,Weighted_MC_2050,Weighted_MC_2035_M95,Weighted_MC_2050_M95,Weighted_LREC_2035,Weighted_LREC_2050,Weighted_Miles_Per_Household,Weighted_EVemission2,Average_Emissions,Weighted_MC_2035_2,Weighted_MC_2050_2,Weighted_MC_2035_M95_2,Weighted_MC_2050_M95_2,Weighted_LREC_2035_2,Weighted_LREC_2050_2,Household_No.,Total_emission,Total_EVemission,Max_change_percentage,Max_change_percentage_EV,Max_change,Total_MC_2035,Total_MC_2050,Total_MC_2035_M95,Total_MC_2050_M95,Total_LREC_2035,Total_LREC_2050
0,AK,3.753831e+09,2.310424e+05,6.634708e+05,2.530491e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,16247.372244,2.871641,10.952496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,255173,2.794781e+06,7.327653e+05,73.780943,73.780943,2.062016e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,AL,3.930070e+10,1.818276e+06,5.665847e+06,2.468316e+07,4.001114e+06,2.597214e+06,3.990519e+06,4.145477e+05,2.654165e+06,1.678057e+06,21614.261662,3.116054,13.575032,2.200498,1.428394,2.194671,0.227989,1.459715,0.922884,1888504,2.563650e+07,5.884680e+06,77.045698,77.045698,1.975182e+07,4.155650e+06,2.697527e+06,4.144645e+06,4.305589e+05,2.756677e+06,1.742869e+06
2,AR,2.531543e+10,1.248170e+06,5.287501e+06,1.664548e+07,7.908514e+05,9.683024e+05,1.277135e+06,5.630657e+05,3.719646e+05,4.018242e+05,20282.035675,4.236203,13.335906,0.633609,0.775778,1.023206,0.451113,0.298008,0.321931,1170544,1.561026e+07,4.958662e+06,68.234608,68.234608,1.065160e+07,7.416669e+05,9.080820e+05,1.197708e+06,5.280477e+05,3.488315e+05,3.768340e+05
3,AZ,4.699380e+10,2.456211e+06,6.533389e+06,2.872644e+07,1.664459e+06,9.201244e+05,1.517176e+06,2.708112e+05,5.416223e+05,4.814421e+05,19132.634933,2.659946,11.695427,0.677653,0.374611,0.617689,0.110256,0.220511,0.196010,2643430,3.091604e+07,7.031380e+06,77.256532,77.256532,2.388466e+07,1.791328e+06,9.902586e+05,1.632819e+06,2.914531e+05,5.829061e+05,5.181388e+05
4,CA,2.656740e+11,1.289250e+07,2.439202e+07,1.516012e+08,2.963514e+06,1.530999e+06,2.694917e+06,1.468327e+06,1.656344e+06,1.163918e+06,20606.857572,1.891954,11.758863,0.229863,0.118751,0.209030,0.113890,0.128473,0.090279,13103114,1.540777e+08,2.479048e+07,83.910404,83.910404,1.292872e+08,3.011925e+06,1.556010e+06,2.738941e+06,1.492313e+06,1.683402e+06,1.182931e+06
5,CO,3.960567e+10,1.959026e+06,9.267378e+06,2.500056e+07,3.550331e+05,1.655042e+05,3.884009e+05,1.067769e+04,1.441488e+05,2.002066e+04,20217.021411,4.730605,12.761732,0.181229,0.084483,0.198262,0.005451,0.073582,0.010220,2137402,2.727695e+07,1.011121e+07,62.931321,62.931321,1.716575e+07,3.873601e+05,1.805739e+05,4.237662e+05,1.164993e+04,1.572740e+05,2.184362e+04
6,CT,2.313515e+10,1.190031e+06,2.290293e+06,1.268555e+07,1.502394e+06,1.634156e+06,1.581139e+06,9.005009e+05,1.157007e+06,1.295786e+06,19440.797950,1.924567,10.659852,1.262483,1.373205,1.328654,0.756704,0.972250,1.088867,1385437,1.476855e+07,2.666366e+06,81.945653,81.945653,1.210219e+07,1.749091e+06,1.902489e+06,1.840767e+06,1.048366e+06,1.346991e+06,1.508557e+06
7,DC,3.814152e+09,4.740136e+05,4.756093e+05,2.097981e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8046.503665,1.003366,4.425993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,288307,1.276045e+06,2.892776e+05,77.330140,77.330140,9.867673e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,DE,8.159160e+09,3.509627e+05,1.356500e+06,4.645619e+06,8.947318e+05,1.059985e+06,9.412007e+05,3.283066e+05,5.719245e+05,6.590880e+05,23247.941664,3.865084,13.236789,2.549365,3.020221,2.681769,0.935446,1.629588,1.877943,370953,4.910227e+06,1.433765e+06,70.800439,70.800439,3.476462e+06,9.456945e+05,1.120360e+06,9.948101e+05,3.470065e+05,6.045005e+05,6.966286e+05
9,FL,1.414728e+11,7.030199e+06,2.263930e+07,8.169559e+07,5.983379e+06,5.649645e+06,7.160985e+06,2.164505e+06,4.238426e+06,3.1180

In [ ]:
# List of total emission columns to calculate row-wise percentages
total_cols = [
    'Total_emission',
    'Total_EVemission',
    'Total_MC_2035',
    'Total_MC_2050',
    'Total_MC_2035_M95',
    'Total_MC_2050_M95',
    'Total_LREC_2035',
    'Total_LREC_2050'
]

# Loop through each total column and create a new column with percentages
for col in total_cols:
    merged_df3[f'{col}_pct'] = (merged_df3[col] / merged_df3[col].sum()) * 100
display(merged_df3)

,HHSTATE_x,Weighted_Miles,WTHHFIN_x,Weighted_EVemission,Weighted_emission,Weighted_MC_2035,Weighted_MC_2050,Weighted_MC_2035_M95,Weighted_MC_2050_M95,Weighted_LREC_2035,Weighted_LREC_2050,Weighted_Miles_Per_Household,Weighted_EVemission2,Average_Emissions,Weighted_MC_2035_2,Weighted_MC_2050_2,Weighted_MC_2035_M95_2,Weighted_MC_2050_M95_2,Weighted_LREC_2035_2,Weighted_LREC_2050_2,Household_No.,Total_emission,Total_EVemission,Max_change_percentage,Max_change_percentage_EV,Max_change,Total_MC_2035,Total_MC_2050,Total_MC_2035_M95,Total_MC_2050_M95,Total_LREC_2035,Total_LREC_2050,Total_emission_pct,Total_EVemission_pct,Total_MC_2035_pct,Total_MC_2050_pct,Total_MC_2035_M95_pct,Total_MC_2050_M95_pct,Total_LREC_2035_pct,Total_LREC_2050_pct
0,AK,3.753831e+09,2.310424e+05,6.634708e+05,2.530491e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,16247.372244,2.871641,10.952496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,255173,2.794781e+06,7.327653e+05,73.780943,73.780943,2.062016e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.184426,0.188355,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,AL,3.930070e+10,1.818276e+06,5.665847e+06,2.468316e+07,4.001114e+06,2.597214e+06,3.990519e+06,4.145477e+05,2.654165e+06,1.678057e+06,21614.261662,3.116054,13.575032,2.200498,1.428394,2.194671,0.227989,1.459715,0.922884,1888504,2.563650e+07,5.884680e+06,77.045698,77.045698,1.975182e+07,4.155650e+06,2.697527e+06,4.144645e+06,4.305589e+05,2.756677e+06,1.742869e+06,1.691738,1.512637,3.748020,3.265960,3.423915,1.413271,4.835607,4.144765
2,AR,2.531543e+10,1.248170e+06,5.287501e+06,1.664548e+07,7.908514e+05,9.683024e+05,1.277135e+06,5.630657e+05,3.719646e+05,4.018242e+05,20282.035675,4.236203,13.335906,0.633609,0.775778,1.023206,0.451113,0.298008,0.321931,1170544,1.561026e+07,4.958662e+06,68.234608,68.234608,1.065160e+07,7.416669e+05,9.080820e+05,1.197708e+06,5.280477e+05,3.488315e+05,3.768340e+05,1.030113,1.274607,0.668916,1.099436,0.989434,1.733269,0.611900,0.896159
3,AZ,4.699380e+10,2.456211e+06,6.533389e+06,2.872644e+07,1.664459e+06,9.201244e+05,1.517176e+06,2.708112e+05,5.416223e+05,4.814421e+05,19132.634933,2.659946,11.695427,0.677653,0.374611,0.617689,0.110256,0.220511,0.196010,2643430,3.091604e+07,7.031380e+06,77.256532,77.256532,2.388466e+07,1.791328e+06,9.902586e+05,1.632819e+06,2.914531e+05,5.829061e+05,5.181388e+05,2.040132,1.807393,1.615616,1.198929,1.348881,0.956669,1.022501,1.232200
4,CA,2.656740e+11,1.289250e+07,2.439202e+07,1.516012e+08,2.963514e+06,1.530999e+06,2.694917e+06,1.468327e+06,1.656344e+06,1.163918e+06,20606.857572,1.891954,11.758863,0.229863,0.118751,0.209030,0.113890,0.128473,0.090279,13103114,1.540777e+08,2.479048e+07,83.910404,83.910404,1.292872e+08,3.011925e+06,1.556010e+06,2.738941e+06,1.492313e+06,1.683402e+06,1.182931e+06,10.167503,6.372311,2.716484,1.883898,2.262655,4.898386,2.952929,2.813161
5,CO,3.960567e+10,1.959026e+06,9.267378e+06,2.500056e+07,3.550331e+05,1.655042e+05,3.884009e+05,1.067769e+04,1.441488e+05,2.002066e+04,20217.021411,4.730605,12.761732,0.181229,0.084483,0.198262,0.005451,0.073582,0.010220,2137402,2.727695e+07,1.011121e+07,62.931321,62.931321,1.716575e+07,3.873601e+05,1.805739e+05,4.237662e+05,1.164993e+04,1.572740e+05,2.184362e+04,1.799991,2.599052,0.349364,0.218625,0.350076,0.038240,0.275881,0.051947
6,CT,2.313515e+10,1.190031e+06,2.290293e+06,1.268555e+07,1.502394e+06,1.634156e+06,1.581139e+06,9.005009e+05,1.157007e+06,1.295786e+06,19440.797950,1.924567,10.659852,1.262483,1.373205,1.328654,0.756704,0.972250,1.088867,1385437,1.476855e+07,2.666366e+06,81.945653,81.945653,1.210219e+07,1.749091e+06,1.902489e+06,1.840767e+06,1.048366e+06,1.346991e+06,1.508557e+06,0.974569,0.685380,1.577522,2.303388,1.520668,3.441167,2.362815,3.587541
7,DC,3.814152e+09,4.740136e+05,4.756093e+05,2.097981e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8046.503665,1.003366,4.425993,0.000000,0.000000,0.000000

In [ ]:
# Step 1: Remove AK, DC, HI
filtered_df = merged_df3[~merged_df3['HHSTATE_x'].isin(['AK', 'DC', 'HI'])].copy()

# Step 2: List of total emission columns
total_cols = [
    'Total_emission',
    'Total_EVemission',
    'Total_MC_2035',
    'Total_MC_2050',
    'Total_MC_2035_M95',
    'Total_MC_2050_M95',
    'Total_LREC_2035',
    'Total_LREC_2050'
]

# Step 3: Create a new DataFrame for percentages
percentage_df = filtered_df[['HHSTATE_x'] + total_cols].copy()

# Step 4: Calculate percentage contributions
for col in total_cols:
    percentage_df[f'{col}_pct'] = (percentage_df[col] / percentage_df[col].sum()) * 100



,HHSTATE_x,Total_emission,Total_EVemission,Total_MC_2035,Total_MC_2050,Total_MC_2035_M95,Total_MC_2050_M95,Total_LREC_2035,Total_LREC_2050,Total_emission_pct,Total_EVemission_pct,Total_MC_2035_pct,Total_MC_2050_pct,Total_MC_2035_M95_pct,Total_MC_2050_M95_pct,Total_LREC_2035_pct,Total_LREC_2050_pct
1,AL,2.563650e+07,5.884680e+06,4.155650e+06,2.697527e+06,4.144645e+06,4.305589e+05,2.756677e+06,1.742869e+06,1.702167,1.526696,3.748020,3.265960,3.423915,1.413271,4.835607,4.144765
2,AR,1.561026e+07,4.958662e+06,7.416669e+05,9.080820e+05,1.197708e+06,5.280477e+05,3.488315e+05,3.768340e+05,1.036463,1.286453,0.668916,1.099436,0.989434,1.733269,0.611900,0.896159
3,AZ,3.091604e+07,7.031380e+06,1.791328e+06,9.902586e+05,1.632819e+06,2.914531e+05,5.829061e+05,5.181388e+05,2.052709,1.824190,1.615616,1.198929,1.348881,0.956669,1.022501,1.232200
4,CA,1.540777e+08,2.479048e+07,3.011925e+06,1.556010e+06,2.738941e+06,1.492313e+06,1.683402e+06,1.182931e+06,10.230182,6.431534,2.716484,1.883898,2.262655,4.898386,2.952929,2.813161
5,CO,2.727695e+07,1.011121e+07,3.873601e+05,1.805739e+05,4.237662e+05,1.164993e+04,1.572740e+05,2.184362e+04,1.811087,2.623207,0.349364,0.218625,0.350076,0.038240,0.275881,0.051947
6,CT,1.476855e+07,2.666366e+06,1.749091e+06,1.902489e+06,1.840767e+06,1.048366e+06,1.346991e+06,1.508557e+06,0.980576,0.691750,1.577522,2.303388,1.520668,3.441167,2.362815,3.587541
8,DE,4.910227e+06,1.433765e+06,9.456945e+05,1.120360e+06,9.948101e+05,3.470065e+05,6.045005e+05,6.966286e+05,0.326021,0.371970,0.852931,1.356447,0.821818,1.139018,1.060380,1.656672
9,FL,9.216713e+07,2.554115e+07,6.750314e+06,6.373803e+06,8.078862e+06,2.441946e+06,4.781697e+06,3.517694e+06,6.119551,6.626285,6.088172,7.716913,6.673994,8.015471,8.387782,8.365523
10,GA,5.249147e+07,1.246667e+07,3.809736e+06,1.661172e+06,4.775869e+06,1.116100e+06,1.176663e+06,1.006509e+06,3.485236,3.234297,3.436037,2.011220,3.945373,3.663498,2.064036,2.393606
12,IA,1.810543e+07,4.184855e+06,9.083067e+05,5.118161e+05,1.049366e+06,1.334344e+04,5.060974e+05,6.767028e+04,1.202132,1.085700,0.819210,0.619668,0.866887,0.043799,0.887767,0.160929


In [ ]:
# Keep only the specified columns
final_columns = [
    'HHSTATE_x',
    'Total_emission_pct',
    'Total_EVemission_pct',
    'Total_MC_2035_pct',
    'Total_MC_2050_pct',
    'Total_MC_2035_M95_pct',
    'Total_MC_2050_M95_pct',
    'Total_LREC_2035_pct',
    'Total_LREC_2050_pct'
]

final_df = percentage_df[final_columns].copy()




,HHSTATE_x,Total_emission_pct,Total_EVemission_pct,Total_MC_2035_pct,Total_MC_2050_pct,Total_MC_2035_M95_pct,Total_MC_2050_M95_pct,Total_LREC_2035_pct,Total_LREC_2050_pct
1,AL,1.702167,1.526696,3.748020,3.265960,3.423915,1.413271,4.835607,4.144765
2,AR,1.036463,1.286453,0.668916,1.099436,0.989434,1.733269,0.611900,0.896159
3,AZ,2.052709,1.824190,1.615616,1.198929,1.348881,0.956669,1.022501,1.232200
4,CA,10.230182,6.431534,2.716484,1.883898,2.262655,4.898386,2.952929,2.813161
5,CO,1.811087,2.623207,0.349364,0.218625,0.350076,0.038240,0.275881,0.051947
6,CT,0.980576,0.691750,1.577522,2.303388,1.520668,3.441167,2.362815,3.587541
8,DE,0.326021,0.371970,0.852931,1.356447,0.821818,1.139018,1.060380,1.656672
9,FL,6.119551,6.626285,6.088172,7.716913,6.673994,8.015471,8.387782,8.365523
10,GA,3.485236,3.234297,3.436037,2.011220,3.945373,3.663498,2.064036,2.393606
12,IA,1.202132,1.085700,0.819210,0.619668,0.866887,0.043799,0.887767,0.160929
